In [1]:
from sentiment import df_sentiment
from candidates_class import df_candidates
from Politweet import get_tweets

In [2]:
data = get_tweets('./datasets/tweets.tsv')
tweets = df_candidates(df_sentiment(data))